In [ ]:
import re
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')

In [9]:
def read_file(filepath):
    
    with open(filepath) as f:
        x=f.read()
        return x

In [10]:
a=read_file("moby_dick_four_chapters.txt")
#a=a.decode('utf-8') #to convert the file into string if it it is not one

In [12]:
import re
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')

tokens= re.sub('[^a-zA-Z]', ' ', a)
tokens = tokens.lower()
tokens = tokens.split()


[nltk_data] Downloading package punkt to C:\Users\Thasin Abedin
[nltk_data]     Orvee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Thasin Abedin
[nltk_data]     Orvee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# organize into sequences of tokens of 25, leaving the 1st token of every precious token
train_len = 25+1 # 25 training words , then one target word

# Empty list of sequences
text_sequences = []

for i in range(train_len, len(tokens)):
    
    # Grab train_len# amount of characters
    seq = tokens[i-train_len:i]
    
    # Add to list of sequences
    text_sequences.append(seq)

In [16]:
' '.join(text_sequences[0]) #demo of taking 25 words removing the first word everytime

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [17]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [22]:
##Keras Tokenization
from keras.preprocessing.text import Tokenizer

tokenizer=Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences=tokenizer.texts_to_sequences(text_sequences) #GIVES NUMERIC ID SEQUENCES OF THE TOKENS

In [27]:
##TO SEE ID VALUES OF TOKENS USE TOKENIZER.INDEX_WORD
#tokenizer.index_word
#tokenizer.word_counts counts the no. of times one word showed up
for i in sequences[0]:
    print(f'{i} : {tokenizer.index_word[i]}') #NUMERIC ID VALUES FOR THE TOKENS WE HAVE

960 : call
14 : me
263 : ishmael
51 : some
261 : years
411 : ago
87 : never
221 : mind
129 : how
111 : long
958 : precisely
260 : having
50 : little
43 : or
37 : no
320 : money
7 : in
23 : my
556 : purse
3 : and
150 : nothing
259 : particular
6 : to
2690 : interest
14 : me
25 : on


In [28]:
tokenizer.word_counts

OrderedDict([('call', 27),
             ('me', 2471),
             ('ishmael', 133),
             ('some', 758),
             ('years', 135),
             ('ago', 84),
             ('never', 449),
             ('mind', 164),
             ('how', 321),
             ('long', 374),
             ('precisely', 37),
             ('having', 142),
             ('little', 767),
             ('or', 950),
             ('no', 1029),
             ('money', 120),
             ('in', 5673),
             ('my', 1812),
             ('purse', 71),
             ('and', 9646),
             ('nothing', 281),
             ('particular', 152),
             ('to', 6497),
             ('interest', 24),
             ('on', 1716),
             ('shore', 26),
             ('i', 7332),
             ('thought', 676),
             ('would', 650),
             ('sail', 104),
             ('about', 1014),
             ('a', 10403),
             ('see', 442),
             ('the', 15592),
             ('watery', 26),
  

In [30]:
#TO KNOW THE NO> OF UNIQUE WORDS IN OUR TEXT FILE
vocabulary_size=len(tokenizer.word_counts)
vocabulary_size

2695

In [32]:
##NOW CONVERT THE SEQUENCES(LIST JEYTA) TO NP.ARRAY
import numpy as np
seq=np.array(sequences)
seq

array([[ 960,   14,  263, ..., 2690,   14,   25],
       [  14,  263,   51, ...,   14,   25,  961],
       [ 263,   51,  261, ...,   25,  961,    5],
       ...,
       [ 956,   12,  167, ...,  262,   53,    2],
       [  12,  167, 2689, ...,   53,    2, 2695],
       [ 167, 2689,    3, ...,    2, 2695,   24]])

In [33]:
#TIME TO MAKE THE X and Y LABELS
from keras.utils import to_categorical

x=seq[:,:-1] #make first 24 words as train and the last word as label in every case
y=seq[:,-1]

#make label to_categorical
y=to_categorical(y, num_classes=vocabulary_size+1) ##the +1 is needed as it is the way keras padding works to hold an extra zero


In [38]:
sequence_length=x.shape[1]  # basically koita kore word thakbe ekekta sequence e, here 25

In [39]:
#MODEL

import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

def create_model(vocabulary_size, sequence_length):
    model = Sequential()
    model.add(Embedding(vocabulary_size, sequence_length, input_length=sequence_length))
    model.add(LSTM(150, return_sequences=True)) ##No of neurons in LSTM is generally multiples of sequence_length
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

model = create_model(vocabulary_size+1, sequence_length) #+1 is because of the way embedding layer works with padding

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            67400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 150)           105600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense_1 (Dense)              (None, 150)               22650     
_________________________________________________________________
dense_2 (Dense)              (None, 2696)              407096    
Total params: 783,346
Trainable params: 783,346
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.fit(x, y, batch_size=128, epochs=100,verbose=1)

Epoch 1/100
11345/11345 [==============================] - 10s 899us/step - loss: 6.2225 - accuracy: 0.0529
Epoch 2/100
11345/11345 [==============================] - 10s 873us/step - loss: 6.1250 - accuracy: 0.0531
Epoch 3/100
11345/11345 [==============================] - 9s 816us/step - loss: 6.0434 - accuracy: 0.0600
Epoch 4/100
11345/11345 [==============================] - 9s 824us/step - loss: 5.9069 - accuracy: 0.0663
Epoch 5/100
11345/11345 [==============================] - 9s 786us/step - loss: 5.8124 - accuracy: 0.0659
Epoch 6/100
11345/11345 [==============================] - 9s 825us/step - loss: 5.7533 - accuracy: 0.0681
Epoch 7/100
11345/11345 [==============================] - 10s 848us/step - loss: 5.6679 - accuracy: 0.0718
Epoch 8/100
11345/11345 [==============================] - 9s 822us/step - loss: 5.5908 - accuracy: 0.0748
Epoch 9/100
11345/11345 [==============================] - 9s 814us/step - loss: 5.5181 - accuracy: 0.0769
Epoch 10/100
11345/11345 [========

In [43]:
# save the model to file
model.save('epoch100.h5')
# save the tokenizer
from pickle import dump,load
dump(tokenizer, open('my_token', 'wb'))

In [ ]:
model.fit(x, y, batch_size=128, epochs=300,verbose=1)

Epoch 1/300
11345/11345 [==============================] - 10s 856us/step - loss: 2.3667 - accuracy: 0.3974
Epoch 2/300
11345/11345 [==============================] - 10s 849us/step - loss: 2.3399 - accuracy: 0.4034
Epoch 3/300
11345/11345 [==============================] - 10s 849us/step - loss: 2.3254 - accuracy: 0.4019
Epoch 4/300
11345/11345 [==============================] - 10s 853us/step - loss: 2.2997 - accuracy: 0.4146
Epoch 5/300
11345/11345 [==============================] - 10s 851us/step - loss: 2.2770 - accuracy: 0.4197
Epoch 6/300
11345/11345 [==============================] - 10s 854us/step - loss: 2.2661 - accuracy: 0.4204
Epoch 7/300
11345/11345 [==============================] - 10s 852us/step - loss: 2.2541 - accuracy: 0.4267
Epoch 8/300
11345/11345 [==============================] - 10s 851us/step - loss: 2.2265 - accuracy: 0.4315
Epoch 9/300
11345/11345 [==============================] - 10s 850us/step - loss: 2.2097 - accuracy: 0.4320
Epoch 10/300
11345/11345 [==

11345/11345 [==============================] - 9s 834us/step - loss: 0.7287 - accuracy: 0.8123
Epoch 152/300
11345/11345 [==============================] - 9s 835us/step - loss: 0.7171 - accuracy: 0.8142
Epoch 153/300
11345/11345 [==============================] - 10s 840us/step - loss: 0.7076 - accuracy: 0.8140
Epoch 154/300
11345/11345 [==============================] - 9s 837us/step - loss: 0.7069 - accuracy: 0.8180
Epoch 155/300
11345/11345 [==============================] - 9s 837us/step - loss: 0.7019 - accuracy: 0.8184
Epoch 156/300
11345/11345 [==============================] - 10s 838us/step - loss: 0.7002 - accuracy: 0.8183
Epoch 157/300
11345/11345 [==============================] - 9s 835us/step - loss: 0.6707 - accuracy: 0.8308
Epoch 158/300
11345/11345 [==============================] - 10s 848us/step - loss: 0.6542 - accuracy: 0.8353
Epoch 159/300
11345/11345 [==============================] - 10s 841us/step - loss: 0.6443 - accuracy: 0.8401
Epoch 160/300
11345/11345 [==

In [ ]:
# save the model to file
model.save('epoch400.h5')
# save the tokenizer
from pickle import dump,load
dump(tokenizer, open('my_token', 'wb'))